### Demo for the ABS2 API Wrapper

The ABS2 API offers a relatively simple interface for a powerful QUBO Solver.
Through this interface, QUBOs may be sent in form of a JSON structure to the QUBO Solver to be solved.

To use the API we have to import from **src** <i>models</i>, as well as the <i>abs2_api</i> class.
The models package contains data models for the sending of requests and the receiving of responses.
The abs2_api package contains the different API calls that may be executed.
Furthermore, to unpack the later needed environment file, we need the <i>json</i> library.

In [47]:
import json

from PIL import Image
import numpy as np

import abs2
from abs2 import ABS2API, ABS2Exception, models

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


To begin communicating with the API, we need to instantiate the ABS2API class.

In [2]:
api = ABS2API()

If no user has been registered with the QUBO solver so far, we can also register one.
To do this, we need a valid email address, a username, first name, last name and affiliation.
We can save all of this information inside a **User** object.

In [3]:
# Before running this cell, replace all the values surrounded by [] with valid usernames, emails etc.
user = models.User(
    username="<your username>",
    email="<your email>",
    firstname="<your first name>",
    lastname="<your last name>",
    affiliation="<your company>",
)

Once we have created the user object, we can now send this user object for registration to the QUBO solver.
The solver should respond either with the status code **201** as a response for successful registration or **409** in case the email is already in use.

In [4]:
try:
    response = api.register_user(user)
except ABS2Exception as e:
    if e.args[0].__contains__("409"):
        print("Status code 409: User already registered")

method=POST, url=https://qubosolver.cs.hiroshima-u.ac.jp/v1/signup, params=None, success=False, status_code=409, message=CONFLICT


Status code 409: User already registered


After successful registration of the user, the server will send a password to the email address provided during the registration process.
The Password contained in the email should then be stored in an environment file **env**.
This environment file should have the following format:
```{"username": "[username]", "password": "[password from email]"}```

In [5]:
f = open("./env", "r")
user_file = json.load(f)
f.close()

After loading the user information into a python object, we can create a simple user model UserNP that simply contains the username and password.

In [6]:
user = models.UserNP(user_file["username"], user_file["password"])

This user object can now be used to retrieve a **Bearer Token** which is used for most future API calls.

In [7]:
tokenMessage = api.retrieve_access_token(user.username, user.password)
token = tokenMessage.access_token

If we had previously uploaded a problem, we can retrieve a list of all problems that are currently stored on the server.

In [8]:
problem_list = api.get_problem_list(token)

In [9]:
problem_list.data

[{'file': 'flower-k.json',
  'bytes': 109666970,
  'time': '2022-12-22 00:25:52',
  'uri_problem': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/problems/flower-k.json'}]

If we haven't uploaded our specific problem yet, we can do so by using the **post_qubo_matrix** method.
This method takes the **Bearer token** from before, as well as the name of a file which contains the problem that is to be solved by the QUBO solver.
The file structure of the file has to be as follows:
```
{
    "file": [name of the file as it will be saved on the server],
    "nbit":32,
    "base":0,
    "qubo":[
        [0,0,2],
        [0,1,2],
        [0,2,-3],
        [0,4,4],
        [1,1,1],
        [1,2,-2],
        [1,3,-2],
        [2,2,-2],
        [2,3,-4],
        [3,3,5],
        [3,4,1],
        [4,4,-4]
        ]
}
```

In this demo we will be using the <i>flower.json.gz</i> problem file to benchmark the QUBO solver.
The problem is designed to obtain a binary image that reproduces a grey scale image <i>flower-k.png</i>.

<center>
<table style="border-collapse:collapse">
    <tr>
        <td><img src="./flower-k.png" alt="flower-k"></td><td><img src="./flower-k-optimal.png" alt="flower-k-optimal"></td>
    </tr>
    <tr>
        <td><center>flower-k</center></td><td><center>flower-k-optimal</center></td>
    </tr>
 </table>
 </center>

By arranging the 65536-bit solution in a 256x256 matrix such that 0/1 are black/white pixels a binary image can be obtained.
The <i>flower-k</i> problem has a known optimal solution with energy -225466781.

In [ ]:
# First we obtain a test dataset

from urllib.request import urlretrieve
import gzip

# download file
sample_file = "https://raw.githubusercontent.com/nakanocs/ABS2WebAPI/main/halftoning/flower-k.json.gz"
urlretrieve(sample_file, "./flower-k.json.gz")

# Decompress using python standard library
with open("./flower-k.json.gz", "rb") as f:
    data = f.read()
with open("./flower-k.json", "wb") as f:
    f.write(gzip.decompress(data))

To upload this problem to the QUBO solver, we make use of the <i>post_qubo_matrix</i> method.

In [10]:
post_response = api.post_qubo_matrix(token, "flower-k.json")

In [11]:
post_response.file

'flower-k.json'

After the upload of the problem file is complete, the server will try to verify the integrity of the problem.
The current progress of the verification can be queried by using the **get_qubo_matrix_information** method.

In [12]:
info_response = api.get_qubo_matrix_information(token, "flower-k.json")
info_response.verify

True

In case we are not sure what exactly the file name on the server side is, we can query to get all currently uploaded problems again:

In [13]:
problem_list = api.get_problem_list(token)
problem_list.data

[{'file': 'flower-k.json',
  'bytes': 109666970,
  'time': '2022-12-22 00:28:00',
  'uri_problem': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/problems/flower-k.json'}]

Once the verification of the problem finishes, the **post_job** method can be used to start the QUBO solver.
The solver will try to find the best solution to the given problem within a given **time_limit**.
As soon as the solver starts a job, the job file will be moved from the job directory to the solution directory on the server.

In [31]:
response = api.post_job(token, "flower-k.json", 600)
response.message

'job for [flower-k.json] submitted successfully'

To assert that the file has been moved and the solver has started working on the problem we can use the **get_all_jobs** method.
Assuming that the job list before only contained the job we had uploaded, this should now be an empty list [].

In [32]:
job_list = api.get_all_jobs(token)
job_list.data

[{'file': 'flower-k_0002.json',
  'bytes': 349,
  'time': '2022-12-22 00:35:29',
  'uri_job': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/jobs/flower-k_0002.json'}]

Finally, we want to see if a viable solution has been found.
The solver works by cycling over all currently active jobs.
If the solver finds a better solution than the current one, the current solution will be overwritten with the better solution.

In [33]:
solution_list = api.get_all_solutions(token)
solution_list.data

[{'file': 'flower-k_0001.json',
  'bytes': 131426,
  'time': '2022-12-22 00:35:37',
  'uri_solution': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/solutions/flower-k_0001.json'}]

After finding a best solution or hitting the hard time limit, the solver stops and the solution can be retrieved by its name with the **get_solution** method.

In [34]:
response = api.get_solution(token, "flower-k_0001.json")

We now have to convert the solution contained in the response into a numpy array

In [ ]:
pixels = np.array(response.solution, dtype=np.uint0)

Since the pixels are currently in the form of a one dimensional array, we have to adjust the shape into a 256x256 array

In [ ]:
pixels = pixels.reshape((256, 256)).T

Next, we create an emtpy, binary image with a size of 256x256 pixels using the <i>Image.new()</i> method and load the pixels

In [ ]:
image = Image.new("1", (256, 256))
image_pixels = image.load()

Once we have loaded the pixels of the image, we iterate over them one by one and replace them with the pixel values that were contained in the solution of the QUBO solver

In [ ]:
for i in range(image.size[0]):
    for j in range(image.size[1]):
        image_pixels[i, j] = int(pixels[i][j])

Lastly we can compare the solution of the solver with the optimal solution:

In [ ]:
image.save("./flower-k-solution.png")

The comparison below shows the solution of the QUBO solver on the left and the optimal solution on the right

<center>
<table style="border-collapse:collapse">
    <tr>
        <td><img src="./flower-k-solution.png" alt="flower-k-solution"></td><td><img src="./flower-k-optimal.png" alt="flower-k-optimal"></td>
    </tr>
    <tr>
        <td><center>flower-k-solution</center></td><td><center>flower-k-optimal</center></td>
    </tr>
 </table>
 </center>

## Upload PyQUBO Problem

In [ ]:
%pip install pyqubo

In [39]:
from pyqubo import Spin

s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
H = (4 * s1 + 2 * s2 + 7 * s3 + s4) ** 2
model = H.compile()
qubo, offset = model.to_qubo()

matrix_response = api.post_pyqubo_matrix(token, qubo=qubo)
matrix_response.message

'[bwewmborce.json] uploaded'

In [44]:
res = api.get_qubo_matrix_information(token, matrix_response.file)
assert res.verify, "Job could not be verified."

In [61]:
job_response = api.post_job(token, matrix_response.file, 600)
job_response.message

'job for [bwewmborce.json] submitted successfully'

In [72]:
api.get_all_jobs(token).data

[{'file': 'bwewmborce_0004.json',
  'bytes': 348,
  'time': '2022-12-22 00:49:21',
  'uri_job': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/jobs/bwewmborce_0004.json'}]

In [73]:
api.get_all_solutions(token).data

[{'file': 'bwewmborce_0003.json',
  'bytes': 415,
  'time': '2022-12-22 00:50:23',
  'uri_solution': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/solutions/bwewmborce_0003.json'},
 {'file': 'flower-k_0001.json',
  'bytes': 131471,
  'time': '2022-12-22 00:39:09',
  'uri_solution': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/solutions/flower-k_0001.json'},
 {'file': 'flower-k_0002.json',
  'bytes': 131472,
  'time': '2022-12-22 00:50:18',
  'uri_solution': 'https://qubosolver.cs.hiroshima-u.ac.jp/v1/solutions/flower-k_0002.json'}]

In [83]:
sol_response = api.get_solution(token, "bwewmborce_0003.json")
sol = matrix_response.decode_solution(sol_response.solution)
sol

{'s2': 0, 's1': 0, 's3': 1, 's4': 0}